<a href="https://colab.research.google.com/github/YogeshKarthik/VAD_using_Autoencoders/blob/main/MiniProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:


import keras
import tensorflow

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:


from tensorflow.keras.utils import img_to_array,load_img
import numpy as np
import glob
import os
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean
import argparse
from PIL import Image
imagestore=[]



video_source_path='/content/drive/MyDrive/Avenue/training_videos'
fps=5
#fps refers to the number of seconds after which one frame will be taken . fps=5 means 1 frame after every 5 seconds. More like seconds per frame.

def create_dir(path):
	if not os.path.exists(path):
		os.makedirs(path)

def remove_old_images(path):
	filelist = glob.glob(os.path.join(path, "*.png"))
	for f in filelist:
		os.remove(f)

def store(image_path):
	img=load_img(image_path)
	img=img_to_array(img)


	#Resize the Image to (227,227,3) for the network to be able to process it.


	img=resize(img,(227,227,3))

	#Convert the Image to Grayscale


	gray=0.2989*img[:,:,0]+0.5870*img[:,:,1]+0.1140*img[:,:,2]

	imagestore.append(gray)



#List of all Videos in the Source Directory.
videos=os.listdir(video_source_path)
print("Found ",len(videos)," training videos")


#Make a temp dir to store all the frames
create_dir(video_source_path+'/frames')

#Remove old images
remove_old_images(video_source_path+'/frames')

framepath=video_source_path+'/frames'

for video in videos:
		os.system( 'ffmpeg -i {}/{} -r 1/{}  {}/frames/%03d.jpg'.format(video_source_path,video,fps,video_source_path))
		images=os.listdir(framepath)
		for image in images:
			image_path=framepath+ '/'+ image
			store(image_path)


imagestore=np.array(imagestore)
a,b,c=imagestore.shape
#Reshape to (227,227,batch_size)
imagestore.resize(b,c,a)
#Normalize
imagestore=(imagestore-imagestore.mean())/(imagestore.std())
#Clip negative Values
imagestore=np.clip(imagestore,0,1)
np.save('trainer.npy',imagestore)
#Remove Buffer Directory
os.system('rm -r {}'.format(framepath))
print("Program ended. Please wait while trainer.npy is created. \nRefresh when needed")
print('Number of frames created :', int(len(imagestore)))

Found  16  training videos
Program ended. Please wait while trainer.npy is created. 
Refresh when needed
Number of frames created : 227


In [ ]:
######DO NOT TOUCH THIS CELL !!!!!!!!!!!!!!!!!!!!!#####################################################################################################################
#This Cell does not include in the main project

from keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np
import argparse
from keras.layers import Conv3D,ConvLSTM2D,Conv3DTranspose, MaxPooling3D, BatchNormalization
from keras.models import Sequential

def load_model():

	model=Sequential()
	model.add(Conv3D(filters=32,kernel_size=(5,5,3),padding='same',input_shape=(227,227,10,1), activation='relu'))
	model.add(MaxPooling3D(pool_size=(2,2,1), strides=(2,2,1)))
	model.add(BatchNormalization())

	model.add(Conv3D(filters=64,kernel_size=(3,3,3),padding='same',activation='relu'))
	model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))
	model.add(BatchNormalization())

	model.add(ConvLSTM2D(filters=64,kernel_size=(3,3),padding='same',dropout=0.3,recurrent_dropout=0.4,return_sequences=True))
	model.add(BatchNormalization())

	model.add(ConvLSTM2D(filters=32,kernel_size=(3,3),padding='same',dropout=0.3,return_sequences=True))
	model.add(BatchNormalization())

	model.add(ConvLSTM2D(filters=64,kernel_size=(3,3),padding='same',dropout=0.5,return_sequences=True))
	model.add(BatchNormalization())

	model.add(Conv3DTranspose(filters=64,kernel_size=(3,3,3),padding='same',activation='relu'))
	model.add(BatchNormalization())

	model.add(Conv3DTranspose(filters=32,kernel_size=(3,3,3),padding='same',activation='relu'))
	model.add(BatchNormalization())

	model.add(Conv3DTranspose(filters=1,kernel_size=(5,5,3),padding='same',activation='sigmoid'))

	model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

	return model

X_train=np.load('trainer.npy')
frames=X_train.shape[2]

# Make the number of frames divisible by 10 to ease the load_model
frames -= frames % 10
X_train=X_train[:,:,:frames]
X_train=X_train.reshape(-1,227,227,10)
X_train=np.expand_dims(X_train,axis=4)
Y_train=X_train.copy()

epochs=300
batch_size=1

if __name__=="__main__":
	model=load_model()

	callback_save = ModelCheckpoint("AnomalyDetector.h5", monitor="val_loss", save_best_only=True, mode='min', verbose=1)
	callback_early_stopping = EarlyStopping(monitor='val_loss', patience=10)

	print('Trainer has been loaded')
	history = model.fit(X_train,Y_train,
			  batch_size=batch_size,
			  epochs=epochs,
			  callbacks = [callback_save,callback_early_stopping],
			  validation_split=0.2,
			  verbose=1
			  )
	print('Done\nPlease wait while AnomalyDetector.h5 has been created \nRefresh when needed')

 #This Cell does not include in the main project
 ##################DO NOT TOUCH THIS CELL!!!!!!!!!!!!!!!!!!!!!!!!!!###########################################################################################################


Trainer has been loaded
Epoch 1/200


ValueError: ignored

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np
import argparse
from keras.layers import Conv3D,ConvLSTM2D,Conv3DTranspose
from keras.models import Sequential



#load_model starts here :----------------------------------------------------
def load_model():

	model=Sequential()
	model.add(Conv3D(filters=128,kernel_size=(11,11,1),strides=(4,4,1),padding='valid',input_shape=(227,227,10,1),activation='tanh'))
	model.add(Conv3D(filters=64,kernel_size=(5,5,1),strides=(2,2,1),padding='valid',activation='tanh'))



	model.add(ConvLSTM2D(filters=64,kernel_size=(3,3),strides=1,padding='same',dropout=0.3,recurrent_dropout=0.4,return_sequences=True))


	model.add(ConvLSTM2D(filters=32,kernel_size=(3,3),strides=1,padding='same',dropout=0.3,return_sequences=True))


	model.add(ConvLSTM2D(filters=64,kernel_size=(3,3),strides=1,return_sequences=True, padding='same',dropout=0.5))




	model.add(Conv3DTranspose(filters=128,kernel_size=(5,5,1),strides=(2,2,1),padding='valid',activation='tanh'))
	model.add(Conv3DTranspose(filters=1,kernel_size=(11,11,1),strides=(4,4,1),padding='valid',activation='tanh'))

	#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
	model.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy'])

	return model

#load_model ends here :----------------------------------------------------



X_train=np.load('trainer.npy')
frames=X_train.shape[2]
#Need to make number of frames divisible by 10 to ease the load_model


frames=frames-frames%10

X_train=X_train[:,:,:frames]
X_train=X_train.reshape(-1,227,227,10)
X_train=np.expand_dims(X_train,axis=4)
Y_train=X_train.copy()


epochs=2000
batch_size=1



if __name__=="__main__":

	model=load_model()

	callback_save = ModelCheckpoint("AnomalyDetector.h5",
									monitor="mean_squared_error")

	callback_early_stopping = EarlyStopping(monitor='val_loss', patience=4)

	print('Trainer has been loaded')
	model.fit(X_train,Y_train,
			  batch_size=batch_size,
			  epochs=epochs,
			  callbacks = [callback_save,callback_early_stopping]
			  )
print('Done\n Please wait while AnomalyDetector.h5 has been created \nRefresh when needed')





FileNotFoundError: ignored

In [ ]:


from tensorflow.keras.utils import img_to_array,load_img
import numpy as np
import glob
import os
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean
import argparse
from PIL import Image
imagestore=[]



video_source_path='/content/drive/MyDrive/Avenue/testing_videos'
fps=5
#fps refers to the number of seconds after which one frame will be taken . fps=5 means 1 frame after every 5 seconds. More like seconds per frame.

def create_dir(path):
	if not os.path.exists(path):
		os.makedirs(path)

def remove_old_images(path):
	filelist = glob.glob(os.path.join(path, "*.png"))
	for f in filelist:
		os.remove(f)

def store(image_path):
	img=load_img(image_path)
	img=img_to_array(img)


	#Resize the Image to (227,227,3) for the network to be able to process it.


	img=resize(img,(227,227,3))

	#Convert the Image to Grayscale


	gray=0.2989*img[:,:,0]+0.5870*img[:,:,1]+0.1140*img[:,:,2]

	imagestore.append(gray)
#List of all Videos in the Source Directory.
videos=os.listdir(video_source_path)
print("Found ",len(videos)," testing videos")


#Make a temp dir to store all the frames
create_dir(video_source_path+'/frames')

#Remove old images
remove_old_images(video_source_path+'/frames')

framepath=video_source_path+'/frames'
total=0
video_count=0

for video in videos:
		video_count+=1
		print("Video number: ",video_count)
		print("Video:",str(video))
		image_count=0
		os.system( 'ffmpeg -i {}/{} -r 1/{}  {}/frames/%03d.jpg'.format(video_source_path,video,fps,video_source_path))
		images=os.listdir(framepath)
		image_count=len(images)
		for image in images:
			image_path=framepath+ '/'+ image
			store(image_path)
		total=len(images)+total
		print("Number of images:",image_count,"\n----------\n")


imagestore=np.array(imagestore)
a,b,c=imagestore.shape
#Reshape to (227,227,batch_size)
imagestore.resize(b,c,a)
#Normalize
imagestore=(imagestore-imagestore.mean())/(imagestore.std())
#Clip negative Values
imagestore=np.clip(imagestore,0,1)
np.save('tester.npy',imagestore)
#Remove Buffer Directory
os.system('rm -r {}'.format(framepath))

print("Program ended. All testing videos shall be stored in tester.npy \n Please wait while tester.npy is created. \nRefresh when needed")
print('Number of frames created :', int(total))
print ('Number of bunches=',int(total),"/10 = ",int(total/10))
print("\nCorrupted and unreadable bunches were ignored")

Found  22  testing videos
Video number:  1
Video: 01.avi
Number of images: 13 
----------

Video number:  2
Video: 02.avi
Number of images: 13 
----------

Video number:  3
Video: 03.avi
Number of images: 13 
----------

Video number:  4
Video: 04.avi
Number of images: 13 
----------

Video number:  5
Video: 05.avi
Number of images: 13 
----------

Video number:  6
Video: 06.avi
Number of images: 13 
----------

Video number:  7
Video: 07.avi
Number of images: 13 
----------

Video number:  8
Video: 08.avi
Number of images: 13 
----------

Video number:  9
Video: 09.avi
Number of images: 13 
----------

Video number:  10
Video: 10.avi
Number of images: 13 
----------

Video number:  11
Video: 11.avi
Number of images: 13 
----------

Video number:  12
Video: 12.avi
Number of images: 13 
----------

Video number:  13
Video: 13.avi
Number of images: 13 
----------

Video number:  14
Video: 14.avi
Number of images: 13 
----------

Video number:  15
Video: 15.avi
Number of images: 13 
-----

In [ ]:

from keras.models import load_model
import numpy as np




def mean_squared_loss(x1,x2):




	diff=x1-x2
	a,b,c,d,e=diff.shape
	n_samples=a*b*c*d*e
	sq_diff=diff**2
	Sum=sq_diff.sum()
	dist=np.sqrt(Sum)
	mean_dist=dist/n_samples

	return mean_dist






'''Define threshold for Sensitivity
Lower the Threshhold,higher the chances that a bunch of frames will be flagged as Anomalous.

'''

threshold=0.0004 #(Accuracy level 1)
#threshold=0.00042 #(Accuracy level 2)
#threshold=0.0008#(Accuracy level Vishakha)

model=load_model('AnomalyDetector.h5')

X_test=np.load('tester.npy')
frames=X_test.shape[2]
#Need to make number of frames divisible by 10


flag=0 #Overall video flagq

frames=frames-frames%10

X_test=X_test[:,:,:frames]
X_test=X_test.reshape(-1,227,227,10)
X_test=np.expand_dims(X_test,axis=4)
counter =0
for number,bunch in enumerate(X_test):
	n_bunch=np.expand_dims(bunch,axis=0)
	reconstructed_bunch=model.predict(n_bunch)


	loss=mean_squared_loss(n_bunch,reconstructed_bunch)

	if loss>threshold:
		print("Anomalous bunch of frames at bunch number {}".format(number))
		counter=counter+1
		print("bunch number: ",counter)
		flag=1


	else:
		print('No anomaly')
		counter=counter+1
		print("bunch number: ",counter)



if flag==1:
	print("Anomalous Events detected")
else:
	print("No anomaly detected")

print("\nCorrupted and unreadable bunches were ignored")

1/1 [==============================] - 1s 1s/step
Anomalous bunch of frames at bunch number 0
bunch number:  1
1/1 [==============================] - 0s 68ms/step
Anomalous bunch of frames at bunch number 1
bunch number:  2
1/1 [==============================] - 0s 72ms/step
No anomaly
bunch number:  3
1/1 [==============================] - 0s 64ms/step
No anomaly
bunch number:  4
1/1 [==============================] - 0s 63ms/step
No anomaly
bunch number:  5
1/1 [==============================] - 0s 62ms/step
No anomaly
bunch number:  6
1/1 [==============================] - 0s 66ms/step
No anomaly
bunch number:  7
1/1 [==============================] - 0s 63ms/step
No anomaly
bunch number:  8
1/1 [==============================] - 0s 61ms/step
Anomalous bunch of frames at bunch number 8
bunch number:  9
1/1 [==============================] - 0s 64ms/step
Anomalous bunch of frames at bunch number 9
bunch number:  10
1/1 [==============================] - 0s 64ms/step
No anomaly
bunch 

In [ ]:

from keras.models import load_model
import numpy as np

from tensorflow.keras.utils import img_to_array,load_img
import numpy as np
import glob
import os
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean
import argparse
from PIL import Image
imagestore=[]



video_source_path='/content/drive/MyDrive/Avenue/testing_videos'
fps=5
#fps refers to the number of seconds after which one frame will be taken . fps=5 means 1 frame after every 5 seconds. More like seconds per frame.

def create_dir(path):
	if not os.path.exists(path):
		os.makedirs(path)

def remove_old_images(path):
	filelist = glob.glob(os.path.join(path, "*.png"))
	for f in filelist:
		os.remove(f)

def store(image_path):
	img=load_img(image_path)
	img=img_to_array(img)


	#Resize the Image to (227,227,3) for the network to be able to process it.


	img=resize(img,(227,227,3))

	#Convert the Image to Grayscale


	gray=0.2989*img[:,:,0]+0.5870*img[:,:,1]+0.1140*img[:,:,2]

	imagestore.append(gray)



#List of all Videos in the Source Directory.
videos=os.listdir(video_source_path)
print("Found ",len(videos)," files")


#Make a temp dir to store all the frames
create_dir(video_source_path+'/frames')

#Remove old images
remove_old_images(video_source_path+'/frames')

framepath=video_source_path+'/frames'
flag=0
for video in videos:
		if (video=="01.avi"):
			print("Test video found")
			flag=1
			os.system( 'ffmpeg -i {}/{} -r 1/{}  {}/frames/%03d.jpg'.format(video_source_path,video,fps,video_source_path))
			images=os.listdir(framepath)
			for image in images:
				image_path=framepath+ '/'+ image
				store(image_path)

if flag==0:
	print("Couldn't find test.mp4 or test.avi. Make sure you reupload and try this")
	exit()
imagestore=np.array(imagestore)
a,b,c=imagestore.shape
#Reshape to (227,227,batch_size)
imagestore.resize(b,c,a)
#Normalize
imagestore=(imagestore-imagestore.mean())/(imagestore.std())
#Clip negative Values
imagestore=np.clip(imagestore,0,1)
np.save('sample.npy',imagestore)
#Remove Buffer Directory
os.system('rm -r {}'.format(framepath))
print("Please wait while video is processed. \nRefresh when needed")


def mean_squared_loss(x1,x2):


	''' Compute Euclidean Distance Loss  between
	input frame and the reconstructed frame'''


	diff=x1-x2
	a,b,c,d,e=diff.shape
	n_samples=a*b*c*d*e
	sq_diff=diff**2
	Sum=sq_diff.sum()
	dist=np.sqrt(Sum)
	mean_dist=dist/n_samples

	return mean_dist


'''Define threshold for Sensitivity
Lower the Threshhold,higher the chances that a bunch of frames will be flagged as Anomalous.

'''

threshold=0.0004 #(Accuracy level 1)
#threshold=0.00042 #(Accuracy level 2)
#threshold=0.0008#(Accuracy level 3)

model=load_model('AnomalyDetector.h5')

X_test=np.load('sample.npy')
frames=X_test.shape[2]
#Need to make number of frames divisible by 10


flag=0 #Overall video flagq

frames=frames-frames%10

X_test=X_test[:,:,:frames]
X_test=X_test.reshape(-1,227,227,10)
X_test=np.expand_dims(X_test,axis=4)
counter =0
for number,bunch in enumerate(X_test):
	n_bunch=np.expand_dims(bunch,axis=0)
	reconstructed_bunch=model.predict(n_bunch)


	loss=mean_squared_loss(n_bunch,reconstructed_bunch)

	if loss>threshold:
		print("Anomalous bunch of frames at bunch number {}".format(number))
		counter=counter+1
		print("bunch number: ",counter)
		flag=1


	else:
		print('No anomaly')
		counter=counter+1
		print("bunch number: ",counter)


print("----------------------------------------------------\nOUTPUT\n----------------------------------------------------\n")
if flag==1:
	print("Anomalous Events detected")
else:
	print("No anomaly detected")

print("\n----------------------------------------------------\nCorrupted and unreadable bunches were ignored")



Found  22  files
Test video found
Please wait while video is processed. 
Refresh when needed
1/1 [==============================] - 1s 683ms/step
Anomalous bunch of frames at bunch number 0
bunch number:  1
----------------------------------------------------
OUTPUT
----------------------------------------------------

Anomalous Events detected

----------------------------------------------------
Corrupted and unreadable bunches were ignored


In [ ]:
import urllib.request

url = "http://www.svcl.ucsd.edu/projects/anomaly/UCSD_Anomaly_Dataset.tar.gz"
filename = "UCSD_Anomaly_Dataset.tar.gz"

urllib.request.urlretrieve(url, filename)

('UCSD_Anomaly_Dataset.tar.gz', <http.client.HTTPMessage at 0x7f0391687b50>)

In [ ]:
import tarfile

filename = "UCSD_Anomaly_Dataset.tar.gz"
output_dir = "/content/drive/MyDrive"

with tarfile.open(filename, "r:gz") as tar:
    tar.extractall(output_dir)

KeyboardInterrupt: ignored

In [ ]:


from tensorflow.keras.utils import img_to_array,load_img
import numpy as np
import glob
import os
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean
import argparse
from PIL import Image
imagestore=[]


for v in range(1,35):
  imagestore=[]
  video_source_path='/content/drive/MyDrive/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Train/'+str(v)
  print(video_source_path)
  fps=5
#fps refers to the number of seconds after which one frame will be taken . fps=5 means 1 frame after every 5 seconds. More like seconds per frame.

  def create_dir(path):
    if not os.path.exists(path):
      os.makedirs(path)

  def remove_old_images(path):
    filelist = glob.glob(os.path.join(path, "*.png"))
    for f in filelist:
      os.remove(f)

  def store(image_path):
	  img=load_img(image_path)
	  img=img_to_array(img)
	#Resize the Image to (227,227,3) for the network to be able to process it.
	  img=resize(img,(227,227,3))
	#Convert the Image to Grayscale
	  gray=0.2989*img[:,:,0]+0.5870*img[:,:,1]+0.1140*img[:,:,2]
	  imagestore.append(gray)
#List of all Videos in the Source Directory.
  videos=os.listdir(video_source_path)
  print("Found ",len(videos)," training videos")


#Make a temp dir to store all the frames
  create_dir(video_source_path+'/frames')

#Remove old images
  remove_old_images(video_source_path+'/frames')

  framepath=video_source_path+'/frames'

  for video in videos:
		  os.system( 'ffmpeg -i {}/{} -r 1/{}  {}/frames/%03d.jpg'.format(video_source_path,video,fps,video_source_path))
		  images=os.listdir(framepath)
		  for image in images:
			  image_path=framepath+ '/'+ image
			  store(image_path)


  imagestore=np.array(imagestore)
  a,b,c=imagestore.shape
#Reshape to (227,227,batch_size)
  imagestore.resize(b,c,a)
#Normalize
  imagestore=(imagestore-imagestore.mean())/(imagestore.std())
#Clip negative Values
  imagestore=np.clip(imagestore,0,1)
  np.save('trainer.npy',imagestore)
#Remove Buffer Directory
  os.system('rm -r {}'.format(framepath))
  print("Program ended. Please wait while trainer.npy is created. \nRefresh when needed")
  print('Number of frames created :', int(len(imagestore)))


/content/drive/MyDrive/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Train/1
Found  202  training videos
Program ended. Please wait while trainer.npy is created. 
Refresh when needed
Number of frames created : 227
/content/drive/MyDrive/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Train/2
Found  200  training videos
Program ended. Please wait while trainer.npy is created. 
Refresh when needed
Number of frames created : 227
/content/drive/MyDrive/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Train/3
Found  200  training videos
Program ended. Please wait while trainer.npy is created. 
Refresh when needed
Number of frames created : 227
/content/drive/MyDrive/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Train/4
Found  200  training videos
Program ended. Please wait while trainer.npy is created. 
Refresh when needed
Number of frames created : 227
/content/drive/MyDrive/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Train/5
Found  200  training videos
Program ended. Please wait while trainer.npy is created. 
Refresh when needed
Number of frames

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np
import argparse
from keras.layers import Conv3D,ConvLSTM2D,Conv3DTranspose
from keras.models import Sequential



def load_model():

	model=Sequential()
	model.add(Conv3D(filters=128,kernel_size=(11,11,1),strides=(4,4,1),padding='valid',input_shape=(227,227,10,1),activation='tanh'))
	model.add(Conv3D(filters=64,kernel_size=(5,5,1),strides=(2,2,1),padding='valid',activation='tanh'))



	model.add(ConvLSTM2D(filters=64,kernel_size=(3,3),strides=1,padding='same',dropout=0.4,recurrent_dropout=0.3,return_sequences=True))


	model.add(ConvLSTM2D(filters=32,kernel_size=(3,3),strides=1,padding='same',dropout=0.3,return_sequences=True))


	model.add(ConvLSTM2D(filters=64,kernel_size=(3,3),strides=1,return_sequences=True, padding='same',dropout=0.5))




	model.add(Conv3DTranspose(filters=128,kernel_size=(5,5,1),strides=(2,2,1),padding='valid',activation='tanh'))
	model.add(Conv3DTranspose(filters=1,kernel_size=(11,11,1),strides=(4,4,1),padding='valid',activation='tanh'))

	model.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy'])

	return model





X_train=np.load('trainer.npy')
frames=X_train.shape[2]
frames=frames-frames%10
X_train=X_train[:,:,:frames]
X_train=X_train.reshape(-1,227,227,10)
X_train=np.expand_dims(X_train,axis=4)
Y_train=X_train.copy()
epochs=200
batch_size=1



if __name__=="__main__":

	model=load_model()

	callback_save = ModelCheckpoint("AnomalyDetector.h5",
									monitor="mean_squared_error")

	callback_early_stopping = EarlyStopping(monitor='val_loss', patience=3)

	print('Trainer has been loaded')
	model.fit(X_train,Y_train,
			  batch_size=batch_size,
			  epochs=epochs,
			  callbacks = [callback_save,callback_early_stopping]
			  )
print('Done\n Please wait while AnomalyDetector.h5 has been created \nRefresh when needed')


Trainer has been loaded
Epoch 1/200
20/20 [==============================] - ETA: 0s - loss: 0.2397 - accuracy: 0.5709

20/20 [==============================] - 14s 289ms/step - loss: 0.2397 - accuracy: 0.5709
Epoch 2/200
20/20 [==============================] - ETA: 0s - loss: 0.2079 - accuracy: 0.5866

20/20 [==============================] - 4s 219ms/step - loss: 0.2079 - accuracy: 0.5866
Epoch 3/200
20/20 [==============================] - ETA: 0s - loss: 0.1731 - accuracy: 0.6235

20/20 [==============================] - 6s 286ms/step - loss: 0.1731 - accuracy: 0.6235
Epoch 4/200
20/20 [==============================] - ETA: 0s - loss: 0.1635 - accuracy: 0.6388

20/20 [==============================] - 6s 282ms/step - loss: 0.1635 - accuracy: 0.6388
Epoch 5/200
20/20 [==============================] - ETA: 0s - loss: 0.1589 - accuracy: 0.6493

...

20/20 [==============================] - 4s 217ms/step - loss: 0.0210 - accuracy: 0.8430
Epoch 197/200
20/20 [==============================] - ETA: 0s - loss: 0.0211 - accuracy: 0.8430

20/20 [==============================] - 6s 292ms/step - loss: 0.0211 - accuracy: 0.8430
Epoch 198/200
20/20 [==============================] - ETA: 0s - loss: 0.0211 - accuracy: 0.8430

20/20 [==============================] - 4s 215ms/step - loss: 0.0211 - accuracy: 0.8430
Epoch 199/200
20/20 [==============================] - ETA: 0s - loss: 0.0210 - accuracy: 0.8430

20/20 [==============================] - 4s 213ms/step - loss: 0.0210 - accuracy: 0.8430
Epoch 200/200
20/20 [==============================] - ETA: 0s - loss: 0.0210 - accuracy: 0.8430

20/20 [==============================] - 6s 285ms/step - loss: 0.0210 - accuracy: 0.8430
Done
 Please wait while AnomalyDetector.h5 has been created 
Refresh when needed


In [ ]:
from tensorflow.keras.utils import img_to_array,load_img
import numpy as np
import glob
import os
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean
import argparse
from PIL import Image
imagestore=[]


for i in range(1,47):
  video_source_path='/content/drive/MyDrive/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test/'+str(i)
  imagestore=[]
  fps=5
#fps refers to the number of seconds after which one frame will be taken . fps=5 means 1 frame after every 5 seconds. More like seconds per frame.

  def create_dir(path):
	  if not os.path.exists(path):
		  os.makedirs(path)

  def remove_old_images(path):
	  filelist = glob.glob(os.path.join(path, "*.png"))
	  for f in filelist:
		  os.remove(f)

  def store(image_path):
	  img=load_img(image_path)
	  img=img_to_array(img)


	#Resize the Image to (227,227,3) for the network to be able to process it.


	  img=resize(img,(227,227,3))

	#Convert the Image to Grayscale


	  gray=0.2989*img[:,:,0]+0.5870*img[:,:,1]+0.1140*img[:,:,2]

	  imagestore.append(gray)
#List of all Videos in the Source Directory.
  videos=os.listdir(video_source_path)
  print("Found ",len(videos)," testing videos")


#Make a temp dir to store all the frames
  create_dir(video_source_path+'/frames')

#Remove old images
  remove_old_images(video_source_path+'/frames')

  framepath=video_source_path+'/frames'
  total=0
  video_count=0

  for video in videos:
		  video_count+=1
		  print("Video number: ",video_count)
		  print("Video:",str(video))
		  image_count=0
		  os.system( 'ffmpeg -i {}/{} -r 1/{}  {}/frames/%03d.jpg'.format(video_source_path,video,fps,video_source_path))
		  images=os.listdir(framepath)
		  image_count=len(images)
		  for image in images:
			  image_path=framepath+ '/'+ image
			  store(image_path)
		  total=len(images)+total
		  print("Number of images:",image_count,"\n----------\n")


  imagestore=np.array(imagestore)
  a,b,c=imagestore.shape
#Reshape to (227,227,batch_size)
  imagestore.resize(b,c,a)
#Normalize
  imagestore=(imagestore-imagestore.mean())/(imagestore.std())
#Clip negative Values
  imagestore=np.clip(imagestore,0,1)
  np.save('tester.npy',imagestore)
#Remove Buffer Directory
  os.system('rm -r {}'.format(framepath))

  print("Program ended. All testing videos shall be stored in tester.npy \n Please wait while tester.npy is created. \nRefresh when needed")
  print('Number of frames created :', int(total))
  print ('Number of bunches=',int(total),"/10 = ",int(total/10))
  print("\nCorrupted and unreadable bunches were ignored")





Streaming output truncated to the last 5000 lines.

Video number:  11
Video: 141.bmp
Number of images: 1 
----------

Video number:  12
Video: 064.bmp
Number of images: 1 
----------

Video number:  13
Video: 140.bmp
Number of images: 1 
----------

Video number:  14
Video: 023.bmp
Number of images: 1 
----------

Video number:  15
Video: 164.bmp
Number of images: 1 
----------

Video number:  16
Video: 056.bmp
Number of images: 1 
----------

Video number:  17
Video: 112.bmp
Number of images: 1 
----------

Video number:  18
Video: 153.bmp
Number of images: 1 
----------

Video number:  19
Video: 116.bmp
Number of images: 1 
----------

Video number:  20
Video: 157.bmp
Number of images: 1 
----------

Video number:  21
Video: 114.bmp
Number of images: 1 
----------

Video number:  22
Video: 063.bmp
Number of images: 1 
----------

Video number:  23
Video: 145.bmp
Number of images: 1 
----------

Video number:  24
Video: 084.bmp
Number of images: 1 
----------

Video number:  25
Video:

In [ ]:
from keras.models import load_model
import numpy as np




def mean_squared_loss(x1,x2):





	diff=x1-x2
	a,b,c,d,e=diff.shape
	n_samples=a*b*c*d*e
	sq_diff=diff**2
	Sum=sq_diff.sum()
	dist=np.sqrt(Sum)
	mean_dist=dist/n_samples

	return mean_dist








threshold=0.00025 #(Accuracy level 1)
#threshold=0.00042 #(Accuracy level 2)
#threshold=0.0008#(Accuracy level Vishakha)

model=load_model('AnomalyDetector.h5')

X_test=np.load('tester.npy')
frames=X_test.shape[2]
#Need to make number of frames divisible by 10


flag=0 #Overall video flagq

frames=frames-frames%10

X_test=X_test[:,:,:frames]
X_test=X_test.reshape(-1,227,227,10)
X_test=np.expand_dims(X_test,axis=4)
counter =0
for number,bunch in enumerate(X_test):
	n_bunch=np.expand_dims(bunch,axis=0)
	reconstructed_bunch=model.predict(n_bunch)


	loss=mean_squared_loss(n_bunch,reconstructed_bunch)

	if loss>threshold:
		print("Anomalous bunch of frames at bunch number {}".format(number))
		counter=counter+1
		print("bunch number: ",counter)
		flag=1


	else:
		print('No anomaly')
		counter=counter+1
		print("bunch number: ",counter)



if flag==1:
	print("Anomalous Events detected")
else:
	print("No anomaly detected")

print("\nCorrupted and unreadable bunches were ignored")

1/1 [==============================] - 1s 706ms/step
No anomaly
bunch number:  1
1/1 [==============================] - 0s 60ms/step
No anomaly
bunch number:  2
1/1 [==============================] - 0s 63ms/step
No anomaly
bunch number:  3
1/1 [==============================] - 0s 62ms/step
No anomaly
bunch number:  4
1/1 [==============================] - 0s 61ms/step
No anomaly
bunch number:  5
1/1 [==============================] - 0s 62ms/step
No anomaly
bunch number:  6
1/1 [==============================] - 0s 60ms/step
No anomaly
bunch number:  7
1/1 [==============================] - 0s 55ms/step
No anomaly
bunch number:  8
1/1 [==============================] - 0s 60ms/step
No anomaly
bunch number:  9
1/1 [==============================] - 0s 60ms/step
No anomaly
bunch number:  10
1/1 [==============================] - 0s 57ms/step
No anomaly
bunch number:  11
1/1 [==============================] - 0s 62ms/step
No anomaly
bunch number:  12
1/1 [==============================]

In [ ]:
!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

rm: cannot remove '/root/.kaggle': No such file or directory


In [ ]:
!kaggle datasets download -d tthien/shanghaitech

 93% 311M/333M [00:01<00:00, 218MB/s]
100% 333M/333M [00:02<00:00, 129MB/s]


In [ ]:


from tensorflow.keras.utils import img_to_array,load_img
import numpy as np
import glob
import os
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean
import argparse
from PIL import Image
imagestore=[]



video_source_path='/content/drive/MyDrive/ShanghaiTech/part_A/train_data/images'
fps=5
#fps refers to the number of seconds after which one frame will be taken . fps=5 means 1 frame after every 5 seconds. More like seconds per frame.

def create_dir(path):
	if not os.path.exists(path):
		os.makedirs(path)

def remove_old_images(path):
	filelist = glob.glob(os.path.join(path, "*.png"))
	for f in filelist:
		os.remove(f)

def store(image_path):
	img=load_img(image_path)
	img=img_to_array(img)


	#Resize the Image to (227,227,3) for the network to be able to process it.


	img=resize(img,(227,227,3))

	#Convert the Image to Grayscale


	gray=0.2989*img[:,:,0]+0.5870*img[:,:,1]+0.1140*img[:,:,2]

	imagestore.append(gray)



#List of all Videos in the Source Directory.
videos=os.listdir(video_source_path)
print("Found ",len(videos)," training videos")


#Make a temp dir to store all the frames
create_dir(video_source_path+'/frames')

#Remove old images
remove_old_images(video_source_path+'/frames')

framepath=video_source_path+'/frames'

for video in videos:
		os.system( 'ffmpeg -i {}/{} -r 1/{}  {}/frames/%03d.jpg'.format(video_source_path,video,fps,video_source_path))
		images=os.listdir(framepath)
		for image in images:
			image_path=framepath+ '/'+ image
			store(image_path)


imagestore=np.array(imagestore)
a,b,c=imagestore.shape
#Reshape to (227,227,batch_size)
imagestore.resize(b,c,a)
#Normalize
imagestore=(imagestore-imagestore.mean())/(imagestore.std())
#Clip negative Values
imagestore=np.clip(imagestore,0,1)
np.save('trainer.npy',imagestore)
#Remove Buffer Directory
os.system('rm -r {}'.format(framepath))
print("Program ended. Please wait while trainer.npy is created. \nRefresh when needed")
print('Number of frames created :', int(len(imagestore)))

Found  300  training videos
Program ended. Please wait while trainer.npy is created. 
Refresh when needed
Number of frames created : 227


In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np
import argparse
from keras.layers import Conv3D,ConvLSTM2D,Conv3DTranspose
from keras.models import Sequential



def load_model():

	"""
	Return the model used for abnormal event
	detection in videos using spatiotemporal autoencoder

	"""
	model=Sequential()
	model.add(Conv3D(filters=128,kernel_size=(11,11,1),strides=(4,4,1),padding='valid',input_shape=(227,227,10,1),activation='tanh'))
	model.add(Conv3D(filters=64,kernel_size=(5,5,1),strides=(2,2,1),padding='valid',activation='tanh'))



	model.add(ConvLSTM2D(filters=64,kernel_size=(3,3),strides=1,padding='same',dropout=0.7,recurrent_dropout=0.3,return_sequences=True))


	model.add(ConvLSTM2D(filters=32,kernel_size=(3,3),strides=1,padding='same',dropout=0.7,return_sequences=True))


	model.add(ConvLSTM2D(filters=64,kernel_size=(3,3),strides=1,return_sequences=True, padding='same',dropout=0.5))




	model.add(Conv3DTranspose(filters=128,kernel_size=(5,5,1),strides=(2,2,1),padding='valid',activation='tanh'))
	model.add(Conv3DTranspose(filters=1,kernel_size=(11,11,1),strides=(4,4,1),padding='valid',activation='tanh'))

	model.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy'])

	return model





X_train=np.load('trainer.npy')
frames=X_train.shape[2]
frames=frames-frames%10
X_train=X_train[:,:,:frames]
X_train=X_train.reshape(-1,227,227,10)
X_train=np.expand_dims(X_train,axis=4)
Y_train=X_train.copy()
epochs=200
batch_size=1



if __name__=="__main__":

	model=load_model()

	callback_save = ModelCheckpoint("AnomalyDetector.h5",
									monitor="mean_squared_error")

	callback_early_stopping = EarlyStopping(monitor='val_loss', patience=3)

	print('Trainer has been loaded')
	model.fit(X_train,Y_train,
			  batch_size=batch_size,
			  epochs=epochs,
			  callbacks = [callback_save,callback_early_stopping]
			  )
print('Done\n Please wait while AnomalyDetector.h5 has been created \nRefresh when needed')


Trainer has been loaded
Epoch 1/200
30/30 [==============================] - ETA: 0s - loss: 0.1857 - accuracy: 0.5554

30/30 [==============================] - 25s 230ms/step - loss: 0.1857 - accuracy: 0.5554
Epoch 2/200
30/30 [==============================] - ETA: 0s - loss: 0.1744 - accuracy: 0.5571

30/30 [==============================] - 7s 232ms/step - loss: 0.1744 - accuracy: 0.5571
Epoch 3/200
30/30 [==============================] - ETA: 0s - loss: 0.1690 - accuracy: 0.5573

30/30 [==============================] - 7s 241ms/step - loss: 0.1690 - accuracy: 0.5573
Epoch 4/200
30/30 [==============================] - ETA: 0s - loss: 0.1679 - accuracy: 0.5573

30/30 [==============================] - 7s 230ms/step - loss: 0.1679 - accuracy: 0.5573
Epoch 5/200
30/30 [==============================] - ETA: 0s - loss: 0.1597 - accuracy: 0.5590

...

30/30 [==============================] - 8s 254ms/step - loss: 0.0966 - accuracy: 0.6479
Epoch 197/200
30/30 [==============================] - ETA: 0s - loss: 0.0964 - accuracy: 0.6479

30/30 [==============================] - 6s 209ms/step - loss: 0.0964 - accuracy: 0.6479
Epoch 198/200
30/30 [==============================] - ETA: 0s - loss: 0.0963 - accuracy: 0.6484

30/30 [==============================] - 8s 252ms/step - loss: 0.0963 - accuracy: 0.6484
Epoch 199/200
30/30 [==============================] - ETA: 0s - loss: 0.0962 - accuracy: 0.6484

30/30 [==============================] - 6s 209ms/step - loss: 0.0962 - accuracy: 0.6484
Epoch 200/200
30/30 [==============================] - ETA: 0s - loss: 0.0966 - accuracy: 0.6478

30/30 [==============================] - 8s 256ms/step - loss: 0.0966 - accuracy: 0.6478
Done
 Please wait while AnomalyDetector.h5 has been created 
Refresh when needed
